In [27]:
import pandas as pd
import os

# Пути к файлам и соответствующие названия колонок
file_paths = {
    # 'bert': '/bert/sim-words5-bert.csv',
    'glove': '/glove_python/sim-words_sent_5_glove.csv',
    'word2vec': '/word2vec/similar_words_sent_5.csv'
}

# Создаем пустой DataFrame с колонкой 'word'
data = pd.DataFrame(columns=['word'])

# Обрабатываем каждый файл
for model_name, path in file_paths.items():
    # Читаем CSV-файл
    full_path = ".." + path
    df = pd.read_csv(full_path)
    
    # Переименовываем колонку Most_Similar_Word в название модели
    df = df.rename(columns={'Most_Similar_Word': model_name})
    
    # Если это первый файл, используем его слова как основу
    if data.empty:
        data['word'] = df['Word']
    
    # Добавляем данные в основной DataFrame
    data[model_name] = df[model_name]

In [28]:
import ast
# Функция для проверки, является ли значение пустым списком или NaN
def is_valid_similar_words(value):
    if pd.isna(value):  # проверяем NaN
        return False
    if isinstance(value, str):  # если данные хранятся как строки (например, "[('слово', 0.5), ...]")
        try:
            lst = ast.literal_eval(value)  # преобразуем строку в список
            return len(lst) > 0  # True, если список не пустой
        except (ValueError, SyntaxError):
            return False
    elif isinstance(value, list):  # если данные уже в формате списка
        return len(value) > 0
    else:
        return False  # на случай других форматов

# Применяем фильтрацию: оставляем строки, где ВСЕ столбцы с похожими словами не пусты
filtered_data = data[
    data['glove'].apply(is_valid_similar_words) &
    data['word2vec'].apply(is_valid_similar_words)
]

In [29]:
def round_similar_words(value):
    if pd.isna(value):
        return value
    if isinstance(value, str):
        try:
            lst = ast.literal_eval(value)
            if isinstance(lst, list):
                rounded = [(word, round(score, 2)) for word, score in lst]
                return str(rounded)
        except (ValueError, SyntaxError):
            return value
    elif isinstance(value, list):
        return [(word, round(score, 2)) for word, score in value]
    return value
filtered_data['glove'] = filtered_data['glove'].apply(round_similar_words)
filtered_data['word2vec'] = filtered_data['word2vec'].apply(round_similar_words)

C:\Users\vadim\AppData\Local\Temp\ipykernel_6480\707709826.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['glove'] = filtered_data['glove'].apply(round_similar_words)
C:\Users\vadim\AppData\Local\Temp\ipykernel_6480\707709826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['word2vec'] = filtered_data['word2vec'].apply(round_similar_words)


In [33]:
n = 10  # кол-во случайных строк

random_sample = filtered_data.sample(n=n)

# Сохраняем в новый CSV-файл
output_file = 'random_sample_similar_words.csv'
random_sample.to_csv(output_file, index=False, encoding='utf-8')

In [31]:
import pandas as pd
docs = pd.read_csv("random_sample_similar_words.csv")
print(docs)

            word                                              glove  \
0           баян                              [('аккордеон', 0.84)]   
1       токарный                                 [('станок', 0.74)]   
2   платоновский        [('гуткинский', 0.72), ('аскинский', 0.71)]   
3      аккуратно                              [('сложённый', 0.77)]   
4        утолить                                 [('печаль', 0.88)]   
..           ...                                                ...   
95      кристалл                               [('фотонный', 0.81)]   
96         ясный                           [('безветренный', 0.74)]   
97        обидно  [('<unk>', 0.82), ('гепатит', 0.77), ('тулярем...   
98        слалом                  [('каяк', 0.82), ('<unk>', 0.72)]   
99      бодрящий        [('ингаляция', 0.72), ('ресурсцентр', 0.7)]   

                                             word2vec  
0   [('флейта', 0.89), ('балалайка', 0.88), ('акко...  
1   [('станок', 0.86), ('чпу', 0.84